In [ ]:
!pip3 install numpy pandas scipy
import os
import numpy as np
import pandas as pd
from scipy import stats
import re

In [ ]:
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install python-terrier tira==0.0.88
else:
    print('We are in the TIRA sandbox.')

In [3]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# this loads and starts pyterrier so that it also works in the TIRA
ensure_pyterrier_is_loaded()

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

/home/paul/git/ir-lab/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Due to execution in TIRA, I have patched ir_datasets to always return the single input dataset mounted to the sandbox.
Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


### Index bauen

In [8]:
data = pt.get_dataset('irds:ir-lab-jena-leipzig-wise-2023/validation-20231104-training')

Load ir_dataset "ir-lab-jena-leipzig-wise-2023/validation-20231104-training" from tira.


In [9]:
print('Build index:')
iter_indexer = pt.IterDictIndexer("/tmp/index", meta={'docno': 100}, verbose=True)
!rm -Rf /tmp/index
indexref = iter_indexer.index(data.get_corpus_iter())
print('Done. Index is created')

Build index:


No settings given in /home/paul/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/paul/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents:   0%|          | 0/61307 [00:00<?, ?it/s]

No settings given in /home/paul/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/paul/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents: 100%|██████████| 61307/61307 [00:31<00:00, 1975.61it/s]


Done. Index is created


### Unsere Retrieval Pipeline

In [10]:
bo1 = pt.rewrite.Bo1QueryExpansion(indexref, verbose=True)
rm3 = pt.rewrite.RM3(indexref, verbose=True)
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25", verbose=True)
tf_idf = pt.BatchRetrieve(indexref, wmodel="TF_IDF", verbose=True)
non_qe_pipeline = tf_idf

In [11]:
qe_test_queryDf = data.get_topics()[["qid","title"]].rename(columns={"title": "query"}).head()

qe_results = [ {"pipe": "bm25_bo1_r",    "df":   (bm25 >> bo1).transform(qe_test_queryDf) },
               {"pipe": "bm25_rm3_r",    "df":   (bm25 >> rm3).transform(qe_test_queryDf) },
               {"pipe": "tf_idf_bo1_r",  "df": (tf_idf >> bo1).transform(qe_test_queryDf) },
               {"pipe": "tf_idf_rm3_r",  "df": (tf_idf >> rm3).transform(qe_test_queryDf) } ]
df = qe_results[0]["df"][["qid", "query_0"]].rename(columns={"query_0": "query"})
for result in qe_results:
    result["df"].rename(columns={"query": result["pipe"]}, inplace=True)
    result["df"].drop(columns=["query_0"], inplace=True)
    df = df.merge(result["df"], on=["qid"])

for r in df.iterrows():
    print(f"""
    qid:          {r[1]["qid"]}
    query:        {r[1]["query"]}
    bm25_bo1_r:   {r[1]["bm25_bo1_r"]}
    bm25_rm3_r:   {r[1]["bm25_rm3_r"]}
    tf_idf_bo1_r: {r[1]["tf_idf_bo1_r"]}
    tf_idf_rm3_r: {r[1]["tf_idf_rm3_r"]}
    """)

No settings given in /home/paul/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/paul/.tira/.tira-settings.json. I will use defaults.
There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


BR(BM25):   0%|          | 0/5 [00:00<?, ?q/s]

Transformer: 100%|██████████| 5/5 [00:00<00:00, 356.30q/s]



    qid:          q072224
    query:        purchase money
    bm25_bo1_r:   applypipeline:off purchas^1.124916191 monei^1.171398406 buyer^0.114867291 seller^0.090024680 length^0.059362351 deposit^0.057799048 properti^0.052173303 cash^0.049139810 sale^0.041757400 estat^0.041346815
    bm25_rm3_r:   applypipeline:off purchas^0.300000012 buyer^0.060062546 agreement^0.028948911 estat^0.035443090 capit^0.036937352 gain^0.027703013 seller^0.058997158 levi^0.027703013 monei^0.300000012 deposit^0.036903422 taxat^0.046171684 launder^0.041129794
    tf_idf_bo1_r: applypipeline:off purchas^1.124916191 monei^1.171398406 buyer^0.114867291 seller^0.090024680 length^0.059362351 deposit^0.057799048 properti^0.052173303 cash^0.049139810 sale^0.041757400 estat^0.041346815
    tf_idf_rm3_r: applypipeline:off purchas^0.300000012 buyer^0.060062546 agreement^0.028948911 estat^0.035443090 capit^0.036937352 gain^0.027703013 seller^0.058997158 levi^0.027703013 monei^0.300000012 deposit^0.036903422 taxat^0.04

In [12]:
topics = data.get_topics()
topics.head()

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,qid,text,title,query,description,narrative
0,q072224,purchase money,purchase money,purchase money,None,None
1,q072226,purchase used car,purchase used car,purchase used car,None,None
2,q072232,buy gold silver,buy gold silver,buy gold silver,None,None
3,q072240,adenovirus,adenovirus,adenovirus,None,None
4,q072242,Water softener,Water softener,water softener,None,None


In [13]:
# Calculation of the number of words in the title
topics["word_count_old"] = topics["title"].map(lambda x: x.count(" ")+1)
# Calculation of the number of characters in the title
topics["length"] = topics["title"].map(lambda x: len(x))

# Calculation of the number of words in the title using the pyterrier tokenizer
# https://pyterrier.readthedocs.io/en/latest/text.html#working-with-passages-rather-than-documents : r'\s+' instead of ' '
topics["word_count"] = topics["title"].map(lambda x: len(re.split(r'\s+', x)))

topics.head()

,qid,text,title,query,description,narrative,word_count_old,length,word_count
0,q072224,purchase money,purchase money,purchase money,None,None,2,14,2
1,q072226,purchase used car,purchase used car,purchase used car,None,None,3,17,3
2,q072232,buy gold silver,buy gold silver,buy gold silver,None,None,3,15,3
3,q072240,adenovirus,adenovirus,adenovirus,None,None,1,10,1
4,q072242,Water softener,Water softener,water softener,None,None,2,14,2


In [14]:
# Calulation of the Threshold
topics["word_count"] = topics["title"].map(lambda x: x.count(" ")+1)
threshold = topics["word_count"].median()
if len(topics[topics["word_count"] > threshold]) > len(topics[topics["word_count"] < threshold]):
    topics["is_long"] = topics["word_count"] > threshold
else:
    topics["is_long"] = topics["word_count"] >= threshold

In [ ]:
print('Create run')
run = non_qe_pipeline.transform(topics)
print('Done, run was created')

In [ ]:
persist_and_normalize_run(run, 'no-query-expansion')